# BERT

In [ ]:
#!pip install transformers -q

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModel.from_pretrained("bert-base-cased")

In [ ]:
import pandas as pd
import numpy as np

data_path = "C:\\Users\\my tech\\Documents\\1. 3A\\NLP\\data\\preprocessed_data.csv"
data = pd.read_csv(data_path)
data.head()


,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,1
1,wonderful little production br br filming tech...,1
2,thought wonderful way spend time hot summer we...,1
3,basically family little boy jake thinks zombie...,0
4,petter mattei love time money visually stunnin...,1


In [ ]:
len(data)

49582

In [ ]:
#!pip install scikit-learn gensim -q

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from gensim.models import Word2Vec
# from torch.utils.data import DataLoader

In [ ]:
# def create_doc_vec(doc_tokens, model, padding_size):
#     vector_size = model.vector_size
#     all_docs = []

#     for tokens in doc_tokens:
#         word_vectors = []

#         for word in tokens:
#             if word in model.wv:
#                 word_vectors.append(model.wv[word])
#             else:
#                 word_vectors.append(np.zeros(vector_size))

#         word_vectors = word_vectors[:padding_size]

#         while len(word_vectors) < padding_size:
#             word_vectors.append(np.zeros(vector_size))

#         all_docs.append(np.array(word_vectors))

#     return np.array(all_docs)

In [ ]:
X = data["review"]
y = data["sentiment"]

x_tokens = [doc.split() for doc in X]

In [ ]:
import tqdm
import torch

In [ ]:
def bert_tokenizer(tokenizer, data, max_length):
  token = {"input_ids": [], "attention_mask": []}

  for sent in data:
    encoded_sent = tokenizer.encode_plus(
        text=sent,
        add_special_tokens=True,
        max_length=max_length,
        truncation=True,
        padding="max_length",
        return_attention_mask=True
      )

    token["input_ids"].append(torch.tensor(encoded_sent["input_ids"]))
    token["attention_mask"].append(torch.tensor(encoded_sent["attention_mask"]))

  token["input_ids"] = torch.stack(token["input_ids"], dim=0)
  token["attention_mask"] = torch.stack(token["attention_mask"], dim=0)
  return token





In [ ]:
def word_embed(device, tokenizer, bert, column, max_length=128, batch_size=8):
  dataloader = DataLoader(dataset=column, batch_size=batch_size, shuffle=False)
  otlst = []

  for batch_data in tqdm.tqdm(dataloader, desc="bert"):
    batch_token = bert_tokenizer(tokenizer, batch_data, max_length)

    for key in batch_token:
      batch_token[key] = batch_token[key].to(device)

    with torch.no_grad(): # model usage when using pytorch no training 1 token: 768 list
      pool_out = bert(**batch_token)
      
      emb = pool_out.last_hidden_state 
    otlst.append(emb)

  sumrslt = torch.cat(otlst, dim=0)
  return sumrslt



In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

result = word_embed(device, tokenizer, model, X)

bert: 100%|██████████| 6198/6198 [05:55<00:00, 17.43it/s]


In [ ]:
Y = torch.tensor(y)

data = {
    "X": result,
    "y": Y
}

In [ ]:
data

{'X': tensor([[-0.5991,  0.3871,  0.9998,  ...,  0.9999, -0.7852,  0.9860],
         [-0.6411,  0.2918,  0.9997,  ...,  0.9999, -0.6456,  0.9784],
         [-0.8663,  0.5999,  1.0000,  ...,  1.0000, -0.5957,  0.9981],
         ...,
         [-0.5383,  0.2810,  0.9995,  ...,  0.9998, -0.7829,  0.9757],
         [-0.6088,  0.3436,  0.9997,  ...,  0.9999, -0.8002,  0.9918],
         [-0.5629,  0.3054,  0.9995,  ...,  0.9998, -0.8059,  0.9747]],
        device='cuda:0'),
 'y': tensor([1, 1, 1,  ..., 0, 0, 0])}

In [ ]:
len(data["X"])


49582

In [ ]:
len(data["y"])

49582

In [ ]:
torch.save(data, "imdb_bert_embed.pt")


In [ ]:
# import numpy as np

# np.save("imdb_bert_embed.npy", result.cpu().numpy())


# BERTS


1. bert_base_cased
2. bert-base-uncased
3. Roberta
4. SBert
5. hateBert
6. albert

In [4]:
from transformers import AutoTokenizer, AutoModel

model_names = {
    "bert_base_cased": "bert-base-cased",
    "bert_base_uncased": "google-bert/bert-base-uncased"
    #"roberta": "FacebookAI/xlm-roberta-base",
    #"sbert": "sentence-transformers/all-MiniLM-L6-v2",
    #"hatebert": "GroNLP/hateBERT",
    #"albert": "albert-base-v2"
}

tokenizers = {}
models = {}

for key, value in model_names.items():
    tokenizers[key] = AutoTokenizer.from_pretrained(value)
    models[key] = AutoModel.from_pretrained(value)



c:\Users\my tech\miniconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\my tech\miniconda3\envs\nlp\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\my tech\miniconda3\envs\nlp\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\m

In [5]:
assert len(tokenizers.values()) == len(models.values())

In [6]:
import pandas as pd
import numpy as np
# from google.colab import drive


# drive.mount("/content/drive")

In [7]:
data_path = "C:\\Users\\my tech\\Documents\\1. 3A\\NLP\\data\\preprocessed_data.csv"

data = pd.read_csv(data_path)
data.head()

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,1
1,wonderful little production br br filming tech...,1
2,thought wonderful way spend time hot summer we...,1
3,basically family little boy jake thinks zombie...,0
4,petter mattei love time money visually stunnin...,1


In [8]:
len(data)

49582

In [9]:
X = data["review"]
y = data["sentiment"]

x_tokens = [doc.split() for doc in X]

In [21]:
# import torch

# torch.save(y, "/content/drive/My Drive/Colab Notebooks/NLP/embedding/y.pt")

In [10]:
import tqdm
import torch
from torch.utils.data import DataLoader

def bert_tokenizer(tokenizer, data, max_length):
    token = {
        "input_ids": [],
        "attention_mask": []
        }

    for sent in data:
        encoded_sent = tokenizer.encode_plus(
            text=sent,
            add_special_tokens=True,
            max_length=max_length,
            truncation=True,
            padding="max_length",
            return_attention_mask=True
        )

        token["input_ids"].append(torch.tensor(encoded_sent["input_ids"]))
        token["attention_mask"].append(torch.tensor(encoded_sent["attention_mask"]))

    token["input_ids"] = torch.stack(token["input_ids"], dim=0)
    token["attention_mask"] = torch.stack(token["attention_mask"], dim=0)
    return token





In [ ]:
def word_embed(device, tokenizer, bert, column, max_length, batch_size=8):
  dataloader = DataLoader(dataset=column, batch_size=batch_size, shuffle=False)
  otlst = []

  for batch_data in tqdm.tqdm(dataloader, desc="bert"):
    batch_token = bert_tokenizer(tokenizer, batch_data, max_length)

    for key in batch_token:
      batch_token[key] = batch_token[key].to(device)

    with torch.no_grad():
        outputs = bert(**batch_token)

        emb = outputs.last_hidden_state
        emb = emb.half()  # float16 болгох (санах ой хэмнэх)
        emb = emb.cpu()   # CPU руу буцаах

    otlst.append(emb)

  sumrslt = torch.cat(otlst, dim=0)
  return sumrslt

In [12]:
import torch

print("torch version:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
print("cuda version:", torch.version.cuda)
print("gpu count:", torch.cuda.device_count())


torch version: 2.2.2
cuda available: True
cuda version: 12.1
gpu count: 1


## PyTorch Upgrade for RTX 5060

RTX 5060-д ажиллуулахын тулд PyTorch-ийг шинэчлэх хэрэгтэй.

**Terminal дээр энэ командыг ажиллуулна уу:**
```bash
pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
```

In [ ]:
# PyTorch-ийг шинэчлэх (энэ нь удаан байж болно)
# Terminal дээр ажиллуулна уу: pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

# Шинэчилсний дараа Kernel-ийг restart хийнэ үү!

In [ ]:
# PyTorch шинэчлэсний дараа GPU ашиглана
device = "cuda:0" if torch.cuda.is_available() else "cpu"

Y = torch.tensor(y.values, dtype=torch.long)
print(f"Using device: {device}")

cuda:0


In [16]:
import os

data_save_path = "C:\\Users\\my tech\\Documents\\1. 3A\\NLP\\data\\embedding\\embedded_x"

if os.path.exists(data_save_path):
    print(True)
else:
    print(False)

False


In [17]:
len(models)

2

In [ ]:
import gc

for key, tokenizer in tokenizers.items():
    model = models[key]
    model = model.to(device)

    print(f"Started embedding: {key}\n")
    result = word_embed(device, tokenizer, model, X, 64)

    print(f"X: {result.shape}")
    print(f"X: {result.dtype}")
    print(result.__len__())

    torch.save(result ,f"{data_save_path}/imdb_{key}_embed.pt")
    print("saved\n")

    del model
    del result

    gc.collect()
    torch.cuda.empty_cache()

c:\Users\my tech\miniconda3\envs\nlp\lib\site-packages\torch\cuda\__init__.py:218: UserWarning: 
NVIDIA GeForce RTX 5060 with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 sm_90.
If you want to use the NVIDIA GeForce RTX 5060 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


c:\Users\my tech\miniconda3\envs\nlp\lib\site-packages\torch\cuda\__init__.py:218: UserWarning: 
NVIDIA GeForce RTX 5060 with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 sm_90.
If you want to use the NVIDIA GeForce RTX 5060 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


Started embedding: bert_base_cased



bert:   0%|          | 0/6198 [00:00<?, ?it/s]


c:\Users\my tech\miniconda3\envs\nlp\lib\site-packages\torch\cuda\__init__.py:218: UserWarning: 
NVIDIA GeForce RTX 5060 with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 sm_90.
If you want to use the NVIDIA GeForce RTX 5060 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


Started embedding: bert_base_cased



bert:   0%|          | 0/6198 [00:00<?, ?it/s]


RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [32]:
data_paths = [
    "/content/drive/MyDrive/Colab Notebooks/NLP/embedding/embedded_x/imdb_bert_base_cased_embed.pt",
    "/content/drive/MyDrive/Colab Notebooks/NLP/embedding/embedded_x/imdb_bert_base_uncased_embed.pt,
    "/content/drive/MyDrive/Colab Notebooks/NLP/embedding/embedded_x/imdb_roberta_embed.pt",
    "/content/drive/MyDrive/Colab Notebooks/NLP/embedding/embedded_x/imdb_sbert_embed.pt",
    "/content/drive/MyDrive/Colab Notebooks/NLP/embedding/embedded_x/imdb_hatebert_embed.pt",
    "/content/drive/MyDrive/Colab Notebooks/NLP/embedding/embedded_x/imdb_albert_embed.pt"
]

for data in data_paths:
    pt = torch.load(data)
    print(f"shape: {pt.shape}")
    print(f"dtype: {pt.dtype}")

SyntaxError: unterminated string literal (detected at line 3) (ipython-input-2131601395.py, line 3)